# Prepare Dataset

In [35]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
df_target = pd.read_csv('../data/target.csv')

In [36]:
print("df_train, df_test, df_target")
print(df_train.shape, df_test.shape, df_target.shape)

for col in df_test.columns:
    if col not in df_train.columns:
        print(f"Column {col} is missing in train set")
        df_train[col] = np.nan 

df_train, df_test, df_target
(7000, 16) (3000, 17) (7000, 1)
Column ID is missing in train set


In [37]:
df_test['ID'].sort_values().head(6)
df_train['ID'] = -1

In [38]:
# tambahkan target ke df_train
df_train['coppaRisk'] = df_target['coppaRisk']
df_test['coppaRisk'] = np.nan
df_train['coppaRisk'].unique()

array([False,  True])

In [39]:
df_train.to_csv('../data/train_with_target.csv', index=False)
df_test.to_csv('../data/test_with_target.csv', index=False)

In [40]:
# merge train and test
df = pd.concat([df_train, df_test], axis=0)
df.shape

(10000, 18)

In [41]:
df.dtypes

developerCountry                    object
countryCode                         object
userRatingCount                      int64
primaryGenreName                    object
downloads                           object
deviceType                          object
hasPrivacyLink                      object
hasTermsOfServiceLink               object
hasTermsOfServiceLinkRating         object
isCorporateEmailScore              float64
adSpent                            float64
appAge                             float64
averageUserRating                  float64
appContentBrandSafetyRating         object
appDescriptionBrandSafetyRating     object
mfaRating                           object
ID                                   int64
coppaRisk                           object
dtype: object

In [42]:
df.describe()

,userRatingCount,isCorporateEmailScore,adSpent,appAge,averageUserRating,ID
count,1.000000e+04,8432.000000,1.895000e+03,9933.000000,8254.000000,10000.000000
mean,3.352463e+04,45.954222,2.859565e+03,56.778133,2.484977,1504.189500
std,5.209562e+05,49.375769,7.699618e+04,36.946823,1.902750,2792.959612
min,0.000000e+00,0.000000,7.038525e-03,-1.200000,0.000000,-1.000000
25%,0.000000e+00,0.000000,1.089968e+00,27.400000,0.000000,-1.000000
50%,1.000000e+00,0.000000,7.750849e+00,52.566667,3.000000,-1.000000
75%,8.525000e+01,99.000000,4.862666e+01,79.600000,4.000000,1680.500000
max,3.854334e+07,99.000000,3.143906e+06,196.466667,5.000000,9999.000000


In [43]:
# Check for missing values
print("Missing values in train set: \n")
print(df_train.isnull().sum())

Missing values in train set: 

developerCountry                      0
countryCode                          64
userRatingCount                       0
primaryGenreName                      0
downloads                          2149
deviceType                            0
hasPrivacyLink                      750
hasTermsOfServiceLink              4635
hasTermsOfServiceLinkRating        4635
isCorporateEmailScore              1128
adSpent                            5679
appAge                               50
averageUserRating                  1232
appContentBrandSafetyRating        6162
appDescriptionBrandSafetyRating       0
mfaRating                             0
ID                                    0
coppaRisk                             0
dtype: int64


# Preprocessing Data

## country code

In [44]:
df['countryCode'].isna().sum()

np.int64(93)

In [45]:
def dummies_by_corr(df, column,top= 10 ,target = 'coppaRisk'):
    df_dummies = pd.get_dummies(df.iloc[:len(df_train)][column], prefix=column, drop_first=True)
    df_dummies[target] = df.iloc[:len(df_train)][target]

    # correlation each column with target
    df_corr = abs(df_dummies.corr())
    df_corr = df_corr[target].sort_values(ascending=False)
    df_corr.pop(target)
    print("Correlation with target:")
    print(df_corr[:10])
    
    # get top 10 columns
    top_10 = df_corr[:top].index.tolist()
    
    df_dummies = pd.get_dummies(df[column], prefix=column, drop_first=True)
    df_dummies = df_dummies[top_10]
    
    # merge with df
    df[top_10] = df_dummies[top_10]
    print("\nShape after merging:")
    print(df.shape)
    
    return df

# df = dummies_by_corr(df, column = 'countryCode', top= 20)

## developer country

In [46]:
df['developerCountry'].nunique()

129

In [47]:
missing_indicators = [
    "", "ADDRESS NOT LISTED IN PLAYSTORE",
    "PERSONAL DATA, CAN NOT BE PUBLICLY DISCLOSED ACCORDING TO APPLICABLE LAWS.",
    "STATUTORY MASKING ENABLED", "CANNOT IDENTIFY COUNTRY"
]
df["developerCountry"] = df["developerCountry"].replace(missing_indicators, np.nan)
df["developerCountry_missing"] = df["developerCountry"].isna().astype(int)

In [48]:
df["developerCountry_missing"].value_counts()

developerCountry_missing
1    7261
0    2739
Name: count, dtype: int64

## downloads

In [49]:
df['downloads'].isna().sum()

np.int64(3030)

In [50]:
# parse downloads to min and max
def parse_downloads(value):
    if pd.isna(value): return (np.nan, np.nan)
    parts = value.replace(',', '').split(' - ')
    return int(parts[0]), int(parts[1])

df[['downloads_min', 'downloads_max']] = df['downloads'].apply(parse_downloads).apply(pd.Series)

In [51]:
df['downloads'].nunique(), df['downloads_min'].nunique(), df['downloads_max'].nunique()

(26, 21, 21)

In [52]:
# Optional: fill missing with median
df['downloads_min'] = df['downloads_min'].fillna(df['downloads_min'].median())
df['downloads_max'] = df['downloads_max'].fillna(df['downloads_max'].median())

df['downloads'] = (df['downloads_min'] + df['downloads_max']) / 2
df['downloads'] = np.log1p(df['downloads']) # log transform

## has privacy & TOS Link

In [53]:
print('null values in each column:')
df[['hasPrivacyLink', 'hasTermsOfServiceLink']].isna().sum()

null values in each column:


hasPrivacyLink           1066
hasTermsOfServiceLink    6596
dtype: int64

In [54]:
# fill with unknown or False

df["hasPrivacyLink"] = df["hasPrivacyLink"].fillna(False) 
df["hasTermsOfServiceLink"] = df["hasTermsOfServiceLink"].fillna(False) 

/tmp/ipykernel_86956/2342464855.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["hasPrivacyLink"] = df["hasPrivacyLink"].fillna(False)
/tmp/ipykernel_86956/2342464855.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["hasTermsOfServiceLink"] = df["hasTermsOfServiceLink"].fillna(False)


## hasTOSRating & appContentBrandSafetyRating

In [55]:
df[['hasTermsOfServiceLinkRating','appContentBrandSafetyRating']].isna().sum()

hasTermsOfServiceLinkRating    6596
appContentBrandSafetyRating    8833
dtype: int64

In [56]:
print("has TOS rating values:",df['hasTermsOfServiceLinkRating'].unique().tolist())
print("app content brand safety rating values:",df['appContentBrandSafetyRating'].unique().tolist())

has TOS rating values: ['low', nan, 'high']
app content brand safety rating values: ['medium', nan, 'low', 'high']


In [57]:
# fill with unknown
df["hasTermsOfServiceLinkRating"] = df["hasTermsOfServiceLinkRating"].fillna("unknown")
df["appContentBrandSafetyRating"] = df["appContentBrandSafetyRating"].fillna("unknown")

## other cols

In [58]:
other_columns = ["isCorporateEmailScore", "adSpent", "appAge", "averageUserRating"]

print("null values in each column:")
df[other_columns].isna().sum()

null values in each column:


isCorporateEmailScore    1568
adSpent                  8105
appAge                     67
averageUserRating        1746
dtype: int64

In [59]:
for col in other_columns:
    df[f"{col}_missing"] = df[col].isna().astype(int)
    df[col] = df[col].fillna(df[col].median())


## Encoder

In [60]:
categorical_columns = ['primaryGenreName', 'deviceType']
ordinal_columns = ['hasTermsOfServiceLinkRating', 'appContentBrandSafetyRating','appDescriptionBrandSafetyRating', 'mfaRating']

# one hot encoding
# for col in categorical_columns:
#     df = pd.get_dummies(df, columns=[col], prefix=col, drop_first=False)

# ordinal encoder
ordinal_mapping = {'low': 0, 'medium': 1, 'high': 2, 'unknown': -1}

for col in ordinal_columns:
    df[col] = df[col].map(ordinal_mapping)


In [61]:
df.select_dtypes(include=['object']).columns.tolist()

['developerCountry',
 'countryCode',
 'primaryGenreName',
 'deviceType',
 'coppaRisk']

In [62]:
df.shape

(10000, 25)

# export result

In [63]:
# export result
df_train = df.iloc[:len(df_train)]
df_test = df.iloc[len(df_train):]

df_train.to_csv('../data/new_train_2_no_encode.csv', index=False)
df_test.to_csv('../data/new_test_2_no_encode.csv', index=False)
df_train.shape, df_test.shape

((7000, 25), (3000, 25))

In [64]:
#check null count for each columns who has > 0 null values
for col in df.columns:
    if df[col].isna().sum() > 0:
        print(f"{col}: {df[col].isna().sum()} null values")

developerCountry: 7261 null values
countryCode: 93 null values
coppaRisk: 3000 null values


# Analysis feature

In [ ]:
import pandas as pd

df_train = pd.read_csv('../data/new_train_2_no_encode.csv')
df_test = pd.read_csv('../data/new_test_2_no_encode.csv')

df_train = df_train.drop(columns=['countryCode','developerCountry'])


## Variation

In [ ]:

df_test.head()

,developerCountry,countryCode,userRatingCount,downloads,hasPrivacyLink,hasTermsOfServiceLink,hasTermsOfServiceLinkRating,isCorporateEmailScore,adSpent,appAge,...,primaryGenreName_Tools,primaryGenreName_Travel,primaryGenreName_Travel & Local,primaryGenreName_Utilities,primaryGenreName_Video Players & Editors,primaryGenreName_Weather,deviceType_connected-tv/ott,deviceType_smartphone,deviceType_tablet,deviceType_undetermined
0,NaN,GLOBAL,2,6.621406,True,False,-1,0.000000,7.750849,81.600000,...,False,False,False,False,False,False,False,False,False,False
1,ICELAND,EMEA,0,8.922792,True,True,0,0.000000,0.027742,24.700000,...,False,False,False,False,False,False,False,False,False,False
2,UNITED STATES,RU,13059,6.621406,True,False,2,99.000000,9.249056,40.300000,...,False,False,False,False,False,False,False,True,False,False
3,NaN,GLOBAL,0,6.621406,True,False,2,0.000000,7.750849,39.233333,...,False,False,False,False,False,False,False,False,False,False
4,NaN,GLOBAL,0,6.621406,False,False,-1,0.000000,7.750849,65.533333,...,False,False,False,False,False,False,False,False,False,False


In [16]:
pd.options.display.float_format = '{:.6f}'.format
df_train.var().sort_values(ascending=False).tail(50)

appAge_missing                             0.007093
countryCode_LATAM                          0.006388
primaryGenreName_Auto & Vehicles           0.006106
developerCountry_SINGAPORE                 0.005965
mfaRating                                  0.005845
countryCode_RU                             0.005824
primaryGenreName_News                      0.005824
primaryGenreName_Video Players & Editors   0.005541
primaryGenreName_Music                     0.005117
primaryGenreName_Photo & Video             0.004834
primaryGenreName_House & Home              0.004551
countryCode_FR                             0.003843
primaryGenreName_Weather                   0.003275
primaryGenreName_Events                    0.003275
countryCode_KR                             0.003275
primaryGenreName_Dating                    0.002991
primaryGenreName_Reference                 0.002991
primaryGenreName_Beauty                    0.002991
countryCode_SA                             0.002991
primaryGenre

## Corelation

In [8]:

abs(df_train.corr()['coppaRisk']).sort_values(ascending=False).drop('coppaRisk').head(20)

primaryGenreName_Games             0.268717
primaryGenreName_Education         0.256343
adSpent_missing                    0.175274
countryCode_GLOBAL                 0.169707
downloads                          0.156582
deviceType_smartphone              0.118975
developerCountry_missing           0.116096
hasTermsOfServiceLinkRating        0.107594
appDescriptionBrandSafetyRating    0.096584
hasTermsOfServiceLink              0.095782
primaryGenreName_Business          0.084266
appContentBrandSafetyRating        0.082043
averageUserRating                  0.080839
developerCountry_CYPRUS            0.074960
primaryGenreName_Lifestyle         0.072855
developerCountry_HONG KONG         0.072134
countryCode_PS                     0.071835
deviceType_tablet                  0.071331
averageUserRating_missing          0.069571
primaryGenreName_Tools             0.068957
Name: coppaRisk, dtype: float64

## Target

In [18]:
df_train['coppaRisk'].value_counts()

coppaRisk
0.000000    6304
1.000000     696
Name: count, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split

X = df_train.drop(columns=['coppaRisk'])
y = df_train['coppaRisk']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

y_train.value_counts(), y_valid.value_counts()

(coppaRisk
 0.000000    5043
 1.000000     557
 Name: count, dtype: int64,
 coppaRisk
 0.000000    1261
 1.000000     139
 Name: count, dtype: int64)